## Main EMR code

In [1]:
project_path = "../../../../../"
utils_path = project_path * "src/utils/"

output_directory = project_path * "data/output/case_1/kerr/";

In [2]:

cd(@__DIR__)
using Pkg; Pkg.activate(project_path); 
Pkg.instantiate();
using OrdinaryDiffEq;
using Optim;
using LineSearches;
using DiffEqFlux;
using DiffEqSensitivity;
using Plots;
using DataFrames;
using CSV;
using Statistics;
using Flux;
using Random;
import BSON: @save, @load
# gr(); # specify backend for plotting

include(utils_path * "utils.jl")
import_project_utils(utils_path);

  Activating project at `~/Escritorio/TFM/code/01_project`


InterruptException: InterruptException:

In [3]:
# specify random seed
seed = 1234;
Random.seed!(seed)

# script conditions
show_plots = true
save_plots_gif = true
save_data = false

# paths
test_name = "test_1_cos/"
model_name = "test_1_cos/" #"encoder/" #

output_dir = output_directory* "models/" * test_name
solutions_dir = output_dir * "solutions/"
metrics_dir = output_directory * "metrics/"
img_dir = output_dir * "train_img_for_gif/"
list_directories = (output_dir, solutions_dir, metrics_dir, img_dir)
create_directories(list_directories)

UndefVarError: UndefVarError: Random not defined

In [4]:
# Define neural network model
NN, NN_params, chain, re = nn_model_case1(model_name);

# --------------------------------------

datasize = 250
mass_ratio = 0.0
dt = 100.0

# TRAIN waveform parameters

χ₀ = Float64(pi); 
ϕ₀ = Float64(0.0); 
# p_space=Float64.([100,50,70,90]); 
p_space = Float64.([100])
M=Float64(1.0); 
# e_space = Float64.([0.2,0.5,0.8])
e_space = Float64.([0.5])
tspan_train = (0, 6.0f4)
a = 0.5

println("Train dataset creation")
train_info = [];
for e in e_space 
    for p in p_space
        train_info_i = get_problem_information_EMR_kerr(χ₀, ϕ₀, p, M, e, a, mass_ratio, tspan_train, datasize, dt);
        push!(train_info, train_info_i)
    end
end

# TEST waveform parameters

χ₀ = Float64(pi); 
ϕ₀ = Float64(0.0); 
p=Float64(80.0); 
M=Float64(1.0); 
e = Float64(0.4)
tspan_test = (tspan_train[2]*48, tspan_train[2]*50)
a=0.3

println("Test dataset creation")
test_info = [get_problem_information_EMR_kerr(χ₀, ϕ₀, p, M, e, a, mass_ratio, tspan_test, datasize, dt)];

# put data in arrays
println("Processing datasets")
datasets = Dict("train" => train_info, "test" => test_info)
processed_data = process_datasets(datasets);


UndefVarError: UndefVarError: nn_model_case1 not defined

Not sure why wave starts from a so high values. I understand that is due to numerical issues

In [5]:
# zero training image

println("Generating zero training steps image")

example = processed_data["train"][1]
zero_training_solution = Array(solve(
    example["nn_problem"],
    RK4(), 
    saveat = example["tsteps"], 
    dt = dt, 
    adaptive=false
))
pred_waveform, _ = compute_waveform(example["dt_data"], zero_training_solution, example["q"], example["M"], example["model_params"])
zero_training_plt = train_plot(example["tsteps"], example["true_waveform"], pred_waveform)

Generating zero training steps image


UndefVarError: UndefVarError: processed_data not defined

In [6]:

train_losses = []
test_losses = []
train_metrics = []
test_metrics = []
plot_list = []

callback_EMR(θ::Vector{Float64}, train_loss::Float64, agregated_metrics::Dict{String, Float64}, train_loss_information::Dict, test_loss_information::Dict; show_plots::Bool = show_plots, save_plots_gif::Bool=save_plots_gif) = begin

    # add losses
    push!(train_losses, agregated_metrics["train_loss"])
    push!(test_losses, agregated_metrics["test_loss"])
    push!(train_metrics, agregated_metrics["train_metric"])
    push!(test_metrics, agregated_metrics["test_metric"])

    # train waveform
    plt1 = train_plot(train_loss_information["tsteps"], train_loss_information["true_waveform"], train_loss_information["pred_waveform"])

    # test waveform
    plt2 = test_plot(test_loss_information["tsteps"], test_loss_information["tsteps"], test_loss_information["true_waveform"], test_loss_information["pred_waveform"])

    l = @layout [a; b]
    plt = plot(plt1, plt2, layout=l)
    if save_plots_gif
        push!(plot_list, plt)
    end

    return false
end


# --------------------------------------

# Train
loss_f(p) = loss_function_case1(p, processed_data=processed_data, batch_number=2)

# optimisers 
# Flux.Optimise.Descent(0.001)
# Flux.Optimise.ADAM(5e-5, (0.9, 0.999)), 
# Flux.Optimise.RADAM(1e-4, (0.9, 0.999)),
# BFGS(initial_stepnorm=0.01, linesearch = LineSearches.BackTracking()),

println("Start Training")
res = DiffEqFlux.sciml_train(
    loss_f, 
    NN_params,
    BFGS(initial_stepnorm=0.01, linesearch = LineSearches.BackTracking()),
    cb=callback_EMR, 
    maxiters = 100
);


LoadError: LoadError: UndefVarError: @layout not defined
in expression starting at /home/rubenbalbastre/Escritorio/TFM/code/01_project/src/processing/experiments/kerr/case_1/EMR_kerr_train.ipynb:22

In [7]:
print(length(plot_list))
plot(plot_list[end])

0

BoundsError: BoundsError: attempt to access 0-element Vector{Any} at index [0]

In [8]:
print("min test loss ", minimum(test_losses))
plot(train_losses, label="train")
plot!(test_losses, label="test")

In [ ]:
# save flux chain models as bson files. To do so, we must save chain model with its parameters
Flux.loadparams!(chain, Flux.params(re(res.minimizer)))
@save solutions_dir*"model_chiphi.bson" chain

# save losses
losses_df = DataFrame(
    epochs = range(1, length(train_losses)),
    test_name=test_name,
    train_loss = train_losses,
    test_loss = test_losses,
    train_metric = train_metrics,
    test_metric = test_metrics,
)

if ! isfile(metrics_dir*"losses.csv")
    CSV.write(metrics_dir*"losses.csv", losses_df)
else
    x = DataFrame(CSV.File(metrics_dir*"losses.csv", types=Dict("test_name" => String31)))
    append!(x, losses_df)
    CSV.write(metrics_dir*"losses.csv", x)
end

In [ ]:
if save_data
    savefig(zero_training_plt, img_dir*"0_train_img.pdf")
    savefig(zero_training_plt, img_dir*"0_train_img.png")
    println("Save train images")
    # save plots
    for (ind, img) in enumerate(plot_list)
        savefig(img, img_dir*string(ind)*"_train_img.pdf")
        savefig(img, img_dir*string(ind)*"_train_img.png")
        if ind == length(plot_list)
            savefig(img, output_dir*"prediction_plot.pdf")
            savefig(img, output_dir*"prediction_plot.png")
        end
    end
end